# Relevanz und Zentralität
In einem Graphen gibt es Knoten, die wichtiger sind als andere. Verschiedene Arten von Relevanz und Methoden zur Bestimmung werden in diesem Abschnitt behandelt.

In [ ]:
import networkx as nx
from tui_dsmt.graph import InteractiveGraph

from graph_data import wiki, wiki_pos

## Inhaltsverzeichnis
- [Arten von Relevanz](#Arten-von-Relevanz)
- [Degree-Zentralität](#Degree-Zentralität)
- [Closeness-Zentralität](#Closeness-Zentralität)
- [Betweenness-Zentralität](#Betweenness-Zentralität)
- [Eigenvector-Zentralität](#Eigenvector-Zentralität)

## Arten von Relevanz
Der folgende **ungerichtete** Graph $G = (V, E)$ enthält als Knoten die Titel verschiedener Wikipedia Artikel aus der Kategorie *Softwarearchitektur* und Kanten zwischen eben diesen Titeln, wenn die zugehörigen Artikel aufeinander mit Hilfe eines Links aufeinander verweisen. Betrachten Sie den Graphen und suchen Sie nach potentiell relevanten Knoten.

In [ ]:
InteractiveGraph(wiki, wiki_pos)

Von Hand nach relevanten Knoten zu suchen entpuppt sich bereits in diesem kleinen Graphen mit circa $1.000$ Knoten als aussichtslos. Wir wollen daher eine Systematik der Relevanz einführen und diese automatisiert berechnen lassen.

Verschiedene Methonde zur Bestimmung der Relevanz werden daher in vier verschiedene Klassen eingeteilt und separat betrachtet:
1. Einfache Algorithmen betrachten den Grad an Verbundenheit der Knoten mit ihren direkten Nachbarn. (Degree)
2. Komplexer ist das Messen der Verbundenheit mit allen anderen Knoten im Netzwerk. (Closeness)
3. Auch die Häufigkeit als Teil eines kürzesten Pfades kann eine Aussage über die Relevanz treffen. (Betweenness)
4. Es existieren weitere Möglichkeiten, den Einfluss eines Knotens auf Grund seiner Verbundenheit mit anderen einflussreichen Knoten einzuschätzen. (Eigenvector)

## Degree-Zentralität
Wie zuvor angedeutet ist die Degree-Zentralität ein Maß für die Anzahl der Verbindungen eines Knotens innerhalb eines Netzwerks. Das Ziel der Verbindung ist dabei nicht relevant, einzig und allein die Anzahl entscheidet. Vergleichen lässt sich das mit einem sozialen Netzwerk: Wer mit vielen anderen Menschen befreundet ist, hat potentiell eine große Reichweite.

Innerhalb eines Graphen lässt sich die Degree-Zentralität $C_D$ für einen spezifischen Knoten $n$ mit Hilfe des Knotengrades bestimmen:

$$
C_D(n) = deg(n)
$$

In [ ]:
degree_per_node = nx.degree(wiki)

degree_per_node['Programmierschnittstelle'], \
degree_per_node['Internet Printing Protocol']

Durch Sortierung lassen sich die im Sinne der Degree-Zentralität relevantesten Knoten finden:

In [ ]:
ds = sorted(((degree_per_node[node], node) for node in wiki.nodes), reverse=True)
ds[:20]

Durch zielgerichtetes Einfärben der Knoten lassen sich auch Muster in der grafischen Darstellung erkennen:

In [ ]:
InteractiveGraph(wiki, wiki_pos, degree_per_node)

## Closeness-Zentralität
Die direkte Reichweite ist allerdings nicht das einzige Kriterium. Wer beispielsweise einen Werbepartner sucht, wird nicht nur auf die direkte Reichweite achten, sondern auch auf die Möglichkeit des "Herumsprechens". Im Sinne der Closeness-Zentralität ist ein Knoten daher relevanter, wenn die durchschnittliche Länge der Verbindungen von ihm zu anderen Knoten relativ kurz ist.

Diese Verbundenheit wird für einen einzelnen Knoten wie folgt berechnet: Von einem Knoten $n$ wird zu jedem anderen Knoten $m$ ein kürzester Pfad gebildet. Die Länge dieser Pfade $l$ wird aufsummiert. Zuletzt wird der Kehrwert gebildet, sodass Knoten mit kurzen Pfaden einen höheren Wert für die Closeness-Zentralität erhalten.

$$
C_C(n) = \frac{1}{\sum_{m \in V, n \neq m} l(n, m)}
$$

In [ ]:
closeness_per_node = nx.closeness_centrality(wiki)

closeness_per_node['Programmierschnittstelle'], \
closeness_per_node['Internet Printing Protocol']

Natürlich lässt sich auch die Closeness-Zentralität für die Sortierung nutzen. Die Ergebnisse weichen bereits bei den oberen Plätzen vom Knotengrad ab.

In [ ]:
cs = sorted(((closeness_per_node[node], node) for node in wiki.nodes), reverse=True)
cs[:20]

In der grafischen Darstellung wird der Unterschied umso deutlicher.

In [ ]:
InteractiveGraph(wiki, wiki_pos, closeness_per_node)

## Betweenness-Zentralität
Die Betweenness-Zentralität hebt die Eigenschaft von Knoten als "Brücken" hervor. Eine Person in einem Netzwerk, die sich auf vielen kürzesten Pfaden befindet, dient als Verbindungsknoten und hat gleichermaßen die Möglichkeit, die Weitergabe von Informationen zu verzögern beziehungsweise die Nutzung von längeren Pfaden zu erzwingen. Die Relevanz ergibt sich also wieder nicht aus der direkten Reichweite, sondern analog zur Closeness-Zentralität durch die Rolle des Knoten im Graphen.

Berechnet wird die Betweenness-Zentralität durch Bildung aller kürzesten Pfade $P(m, o)$ zwischen jedem beliebigen Paar $(m, o)$ zweier unterschiedlicher Knoten. Um den Wert für einen spezifischen Knoten $n$ zu erhalten, wird die Anzahl dieser Pfade, die $n$ enthalten, $P_n(m, o)$ durch die Gesamtanzahl von Pfaden dividiert und für alle Paare aufsummiert.

$$
C_B(n) = \sum_{(m,o) \in V \times V, m \neq o} \frac{P_n(m, o)}{P(m, o)}
$$

In [ ]:
betweenness_per_node = nx.betweenness_centrality(wiki)

betweenness_per_node['Programmierschnittstelle'], \
betweenness_per_node['Internet Printing Protocol']

Erneut weicht die Sortierung nach diesem Kriterium von den anderen Methoden ab.

In [ ]:
bs = sorted(((betweenness_per_node[node], node) for node in wiki.nodes), reverse=True)
bs[:20]

Mit Hilfe des Einfärbens ist der Unterschied auch im Graphen deutlich zu sehen:

In [ ]:
InteractiveGraph(wiki, wiki_pos, betweenness_per_node)

## Eigenvector-Zentralität
Die Eigenvector-Zentralität weist jedem Knoten im Netzwerk eine relative Relevanz zu. Berechnet wird diese auf Basis der Relevanz der Nachbarknoten, sodass nur Knoten relevant sein können, die ihrerseits relevante Nachbarn besitzen. Der Name leitet sich aus der Berechnung ab, die sich auf ein Eigenwertproblem zurückführen lässt.

### PageRank
Einer der bekanntesten Algorithmen zur Einschätzung der Relevanz ist PageRank. Ursprünglich von Larry Page und Sergey Brin entwickelt, diente er der Google Suche in ihrer Anfangszeit als Kriterium für die Bewertung von Websites. Weit oben in den Suchergebnissen sollten die Seiten auftauchen, die nicht nur häufig, sondern insbesondere von anderen als wichtig eingeschätzten Seiten verlinkt wurden.

Zur Berechnung der PageRank-Wertung wurde ursprünglich eine rekursive Formel aufgestellt. Benötigt wird zur Berechnung des Wertes für einen Knoten $n$ die Menge der auf ihn verweisenden Nachbarn $N_n$, die Menge der Nachbarn $S_n$, auf die $n$ verweist, und ein Dämpfungsfaktor $d \in (0, 1)$.

$$
C_{PR}(n) = \frac{1 - d}{|N_n|} + d * \sum_{m \in N_n} \frac{C_{PR}(m)}{|S_m|}
$$

Die Gleichung lässt sich auf ein Eigenwertproblem zurückführen und effizient mit Hilfe von Matrixoperationen lösen. In NetworkX ist eine entsprechende Funktion ebenfalls integriert.

In [ ]:
pagerank_per_node = nx.betweenness_centrality(wiki)

pagerank_per_node['Programmierschnittstelle'], \
pagerank_per_node['Internet Printing Protocol']

Wie bereits zuvor folgen die Knoten mit der höchsten Bewertung:

In [ ]:
prs = sorted(((pagerank_per_node[node], node) for node in wiki.nodes), reverse=True)
prs[:20]

Auch das Einfärben offenbart Unterschiede zu anderen Methoden:

In [ ]:
InteractiveGraph(wiki, wiki_pos, pagerank_per_node)